In [31]:
# Upload fine-tuning files

import os
from openai import AzureOpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://llm-ft.openai.azure.com/"
os.environ["AZURE_OPENAI_KEY"] = "<YOUR_AZURE_API_KEY>"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

In [2]:
training_file_name = './train_data.jsonl'
validation_file_name = './validate_data.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

In [3]:
print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-0d4b770a65664fd1b38658d595458b01
Validation file ID: file-773536a60ebd455d98dda07748bf3435


In [5]:
training_file_id = "file-0d4b770a65664fd1b38658d595458b01"
validation_file_id = "file-773536a60ebd455d98dda07748bf3435"

In [22]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-1106", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
    hyperparameters={
    "n_epochs": 24,
    "batch_size": 32
  }
)

job_id = response.id

In [ ]:
# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.
# e51ca50072d046dfa0ed414b874a97d2
print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

In [ ]:
response = client.fine_tuning.jobs.retrieve("ftjob-ccfe37ea454940c2acb1099d7acf2c3f")

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

In [ ]:
response = client.fine_tuning.jobs.retrieve(" ftjob-27053b12494c439ebb2b85ef39a48916")
if response.status == 'succeeded':
    result_file_id = response.result_files[0]

retrieve = client.files.retrieve(result_file_id)

# Download the result file.
print(f'Downloading result file: {result_file_id}')

with open(retrieve.filename, "wb") as file:
    result = client.files.content(result_file_id).read()
    file.write(result)